In [1]:
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from PIL import Image
from tqdm import tqdm
from function import *
from models import *
import os
import pywt

!nvidia-smi
print("CUDA Available : ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Mon Nov 20 16:17:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:2B:00.0  On |                  N/A |
|  0%   52C    P8              30W / 350W |    642MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Preprocessing
test_tfm = transforms.Compose([
    transforms.Resize([160, 160]),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
toPIL = transforms.ToPILImage()
totensor = transforms.ToTensor()
normal = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [3]:
# Casia
train_set = DatasetFolder("../../dataset/casia100_dataset/casia", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
#train_set_size = int(len(train_set) * 0.8)
#train_set, valid_set = torch.utils.data.random_split(train_set, [train_set_size, len(train_set) - train_set_size])

train_loader = DataLoader(train_set, batch_size=128, shuffle=False, num_workers=0)
#test_loader = DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=0)

Debugger :  class -> idx 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95, '96': 96, '97': 97, '98': 98, '99

In [4]:
model = InceptionResnetV1(pretrained='casia-webface',classify=None).to(device)
model.logits = nn.Linear(in_features = model.logits.in_features, out_features = 100)
model.last_linear = nn.Linear(in_features = model.last_linear.in_features, out_features = 100, bias = True)
model.last_bn = nn.BatchNorm1d(num_features = 100)
checkpoint = torch.load('pretrain/facenet_casia.pth')
model.load_state_dict(checkpoint)
#cudnn.benchmark = True
model.to(device)
adversary = attack(model, 'fgsm')


path = ['adv_cosine_2', 'clean_cosine_2']
level = 10


for _dir_ in path:
    if not os.path.isdir(_dir_):
        os.mkdir(_dir_)
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        result = imgs
        if 'adv' in _dir_:
            imgs, eta = adversary.perturb(imgs, labels)
            result = imgs
        result = result.cpu()
        
        
        # DWT
        #result, (LL, LH, HL, HH) = discrete_wavelet(result, 'db2', 1)
        
        # DCT
        result = dct2(result, 8)
        #result = idct2(result, 8)
        
        print(result[0])
        
        # DCT_Analysis by convolution
        #result = dct2_ana(result, 3)
        #print(result.shape)

        # DCT
        for i in range(imgs.size(0)):
            img = toPIL(torch.clamp(result[i], 0, 1)).convert('RGB')
            img.save(f'{_dir_}/dct_{i}.png')
            
            img = toPIL(torch.clamp(imgs[i], 0, 1)).convert('RGB')
            img.save(f'{_dir_}/img_{i}.png')
        break

  0%|          | 0/125 [00:00<?, ?it/s]

tensor([[[ 1.6162e+00, -4.9338e-01,  1.2498e-01,  ..., -1.6305e-02,
          -7.7842e-03, -7.2462e-03],
         [-1.5404e-01,  1.8855e-01, -4.2107e-02,  ..., -1.7045e-02,
          -7.5423e-03,  1.4311e-02],
         [-9.5046e-02, -2.8700e-02, -1.2203e-02,  ..., -1.5300e-02,
          -8.9109e-03, -9.4949e-03],
         ...,
         [-9.9496e-03,  1.4136e-02, -9.1468e-03,  ..., -1.4064e-02,
          -4.0324e-03, -1.4403e-02],
         [ 6.9054e-03,  1.0706e-02,  6.1839e-03,  ...,  1.0110e-02,
          -2.1653e-02, -1.4456e-02],
         [-5.8145e-03,  1.0806e-02, -2.6891e-02,  ..., -2.6203e-03,
           2.0258e-02,  9.4601e-03]],

        [[ 1.4523e+00, -5.2644e-01,  1.8126e-01,  ...,  2.0193e-02,
           2.0760e-02,  1.1381e-03],
         [-1.5928e-01,  2.0065e-01, -3.9173e-02,  ...,  7.5298e-03,
          -3.3041e-03,  4.8443e-03],
         [-3.9663e-02, -1.2543e-02, -3.2130e-02,  ...,  2.4487e-02,
           7.7122e-03, -1.3868e-02],
         ...,
         [ 9.0377e-03,  1

  0%|          | 0/125 [00:00<?, ?it/s]

tensor([[[ 1.5368e+00, -4.9376e-01,  9.8383e-02,  ...,  3.1419e-04,
           1.0455e-03, -2.3132e-04],
         [-1.6359e-01,  1.8721e-01, -5.5822e-02,  ..., -3.1806e-04,
          -1.8354e-03, -1.4436e-03],
         [-2.7036e-02, -2.7127e-02, -3.1774e-02,  ...,  1.1012e-04,
           6.9324e-04,  2.7050e-04],
         ...,
         [-2.0034e-03, -1.6545e-03, -1.8596e-03,  ..., -1.1690e-03,
           3.3173e-03, -3.2513e-03],
         [ 2.7596e-03,  1.8394e-03,  8.9629e-04,  ...,  1.2432e-03,
           5.1906e-04, -2.1461e-03],
         [ 6.2305e-04, -2.6360e-03,  1.3538e-04,  ..., -2.1155e-03,
           4.1688e-04,  7.8625e-04]],

        [[ 1.4426e+00, -4.9376e-01,  9.8383e-02,  ...,  3.1419e-04,
           1.0456e-03, -2.3128e-04],
         [-1.6359e-01,  1.8721e-01, -5.5822e-02,  ..., -3.1805e-04,
          -1.8354e-03, -1.4436e-03],
         [-2.7036e-02, -2.7127e-02, -3.1774e-02,  ...,  1.1012e-04,
           6.9324e-04,  2.7050e-04],
         ...,
         [-9.0241e-05,  6